# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import pandas as pd
import numpy as np 

import pickle

import re
from sqlalchemy import create_engine

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.base import BaseEstimator,TransformerMixin

import xgboost as xgb

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_eng')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\benedict.grant\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\benedict.grant\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\benedict.grant\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\benedict.grant\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\benedict.grant\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\benedict.grant\AppData\

True

In [2]:
from sqlalchemy import create_engine, inspect

# Create the engine
filepath = "../data/response_db.db"
engine = create_engine('sqlite:///' + filepath)

# Use the inspector to get table names
inspector = inspect(engine)
table_names = inspector.get_table_names()
table_names


['Response_db_table']

In [7]:
filepath = "../data/response_db.db"
engine = create_engine('sqlite:///' + filepath)
df = pd.read_sql_table('Response_db_table',engine)

In [4]:
# load data from database
filepath = "../data/Response_db.db"
engine = create_engine('sqlite:///' + filepath)
df = pd.read_sql_table('DISASTER_RESPONSE_TABLE',engine)
df.head(1)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
df.describe().iloc[:,:20]

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid
count,26216.00000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.0,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000
mean,15224.82133,0.773650,0.170659,0.004501,0.414251,0.079493,0.050084,0.027617,0.017966,0.032804,0.0,0.063778,0.111497,0.088267,0.015449,0.023039,0.011367,0.033377,0.045545,0.131446
std,8826.88914,0.435276,0.376218,0.066940,0.492602,0.270513,0.218122,0.163875,0.132831,0.178128,0.0,0.244361,0.314752,0.283688,0.123331,0.150031,0.106011,0.179621,0.208500,0.337894
min,2.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7446.75000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15662.50000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22924.25000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.00000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [15]:
df.isnull().sum()[:10]

id                      0
message                 0
original            16046
genre                   0
related                 0
request                 0
offer                   0
aid_related             0
medical_help            0
medical_products        0
dtype: int64

In [16]:
# Child alone has 0 for all values so drop 
# Realted has a max of 2, most likley a fluke for a binary input 
# Only Nulls in originl and this will be dropped as its not usefull in this context 
df.drop('child_alone',axis = 1,inplace=True)
df['related'] = df['related'].apply(lambda x: 1 if x > 0 else 0)

In [17]:
X = df['message']
y = df.drop(['id','message','original','genre'],axis = 1)

### 2. Write a tokenization function to process your text data

In [18]:
def tokenize(text):
    """
    Tokenize the disaster response text
    
    Arguments: text -> Input text that requires 
    Output: final_tokens -> tokens from the extracted text
    """

    #For all url's, replace with a place holder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    url_vals = re.findall(url_regex,text)
    for url in url_vals:
        text = text.replace(url, 'urlplacehold')

    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Extract the word tokens from the provided text
    tokens = nltk.word_tokenize(text)
    
    # Initialize lemmatizer and stemmer
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    
    stop_words = set(stopwords.words('english'))
    final_tokens= [stemmer.stem(lemmatizer.lemmatize(w).lower().strip()) for w in tokens if w.lower() not in stop_words]

    return final_tokens

<>:10: SyntaxWarning: invalid escape sequence '\('
<>:10: SyntaxWarning: invalid escape sequence '\('
C:\Users\benedict.grant\AppData\Local\Temp\ipykernel_25512\791811035.py:10: SyntaxWarning: invalid escape sequence '\('
  url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [19]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
            ('tfidf_transformer', TfidfTransformer())
        ]))
    ])),
    ('classifier', MultiOutputClassifier(xgb.XGBClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline_fit= pipeline.fit(X_train, y_train)

c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:523: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\benedict.grant\AppData\Local\Temp\ipykernel_25512\791811035.py:10: SyntaxWarning: invalid escape sequence '\('
  url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'


KeyboardInterrupt: 

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [27]:
#y_predict = pipeline_fit.predict(X_test)

report = classification_report(y_test.values, y_predict, target_names=y.columns.values, output_dict=True, zero_division=0)

weighted_avg = report['weighted avg']
print("Weighted Average Metrics:")
for metric, value in weighted_avg.items():
    print(f"{metric.capitalize()}: {value:.2f}")

Weighted Average Metrics:
Precision: 0.32
Recall: 0.31
F1-score: 0.31
Support: 20704.00


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
parameters = {
    'classifier__estimator__learning_rate': [0.01, 0.1, 0.2],
    'classifier__estimator__n_estimators': [50, 100, 200,500,1000]
}

# number of folds reduced for computation time 
cv = GridSearchCV(pipeline, parameters, verbose=2, n_jobs=1,scoring='f1',cv = 2)
cv.fit(X_train, y_train)

c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:523: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Fitting 2 folds for each of 15 candidates, totalling 30 fits


c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_scorer.py", line 376, in 

[CV] END classifier__estimator__learning_rate=0.01, classifier__estimator__n_estimators=50; total time= 1.5min


c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_scorer.py", line 376, in 

...

c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_scorer.py", line 376, in 

[CV] END classifier__estimator__learning_rate=0.2, classifier__estimator__n_estimators=1000; total time=17.0min


c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_scorer.py", line 376, in 

[CV] END classifier__estimator__learning_rate=0.2, classifier__estimator__n_estimators=1000; total time=17.6min


GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('count_vectorizer',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x000002606DD4F060>)),
                                                                                        ('tfidf_transformer',
                                                                                         TfidfTransformer())]))])),
                                       ('classifier',
                                        MultiOutputClassifier(estimator=XGBClassifier(base_score=None,
                                                                                      booster=None,
                                                                                      callbac...
                                                                                      max_depth=None,
                                                                                      max_leaves=None,
                                                                                      min_child_weight=None,
                                                                                      missing=nan,
                                                                                      monotone_constraints=None,
                                                                                      multi_strategy=None,
                                                                                      n_estimators=None,
                                                                                      n_jobs=None,
                                                                                      num_parallel_tree=None,
                                                                                      random_state=None, ...)))]),
             n_jobs=1,
             param_grid={'classifier__estimator__learning_rate': [0.01, 0.1,
                                                                  0.2],
                         'classifier__estimator__n_estimators': [50, 100, 200,
                                                                 500, 1000]},
             scoring='f1', verbose=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
#performing far worst
y_prediction_cv= cv.predict(X_test)
print(classification_report(y_test.values, y_prediction_cv, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.77      1.00      0.87      5047
               request       0.00      0.00      0.00      1115
                 offer       0.00      0.00      0.00        25
           aid_related       0.82      0.32      0.46      2740
          medical_help       0.00      0.00      0.00       516
      medical_products       0.00      0.00      0.00       312
     search_and_rescue       0.00      0.00      0.00       196
              security       0.00      0.00      0.00       126
              military       0.00      0.00      0.00       217
                 water       0.00      0.00      0.00       409
                  food       0.00      0.00      0.00       722
               shelter       0.00      0.00      0.00       578
              clothing       0.00      0.00      0.00        90
                 money       0.00      0.00      0.00       158
        missing_people       0.00      

c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [13]:
#This class identifies if the first word of each sentence in a text is a verb or ‘RT’ 
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            if pos_tags:  # Check if pos_tags is not empty
                first_word, first_tag = pos_tags[0]
                if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                    return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)
    

pipeline_verb_xgb = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
            ('tfidf_transformer', TfidfTransformer())
        ])),
        ('starting_verb', StartingVerbExtractor())
    ])),
    ('classifier', MultiOutputClassifier(xgb.XGBClassifier()))
])

pipeline_verb_rf = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
            ('tfidf_transformer', TfidfTransformer())
        ])),
        ('starting_verb', StartingVerbExtractor())
    ])),
    ('classifier', MultiOutputClassifier(RandomForestClassifier()))
])

pipeline_verb_ada = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
            ('tfidf_transformer', TfidfTransformer())
        ])),
        ('starting_verb', StartingVerbExtractor())
    ])),
    ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
])

pipelines = {
    'XGB': pipeline_verb_xgb,
    'RandomForest': pipeline_verb_rf,
    'AdaBoost': pipeline_verb_ada
}

# Fit and predict for each pipeline
# Due to computation time each model will be run without the grid search 
for name, pipeline in pipelines.items():
    print(f"Training and evaluating {name} pipeline...")
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    print(f"Classification report for {name} pipeline:")
    print(classification_report(y_test.values, y_pred, target_names=y.columns.values))


c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:523: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Training and evaluating XGB pipeline...
Classification report for XGB pipeline:
                        precision    recall  f1-score   support

               related       0.84      0.94      0.89      5047
               request       0.78      0.55      0.65      1115
                 offer       0.00      0.00      0.00        25
           aid_related       0.78      0.65      0.71      2740
          medical_help       0.60      0.28      0.38       516
      medical_products       0.69      0.35      0.47       312
     search_and_rescue       0.73      0.19      0.31       196
              security       0.33      0.01      0.02       126
              military       0.60      0.33      0.43       217
                 water       0.77      0.67      0.71       409
                  food       0.81      0.73      0.77       722
               shelter       0.75      0.60      0.67       578
              clothing       0.75      0.52      0.61        90
                 money 

c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _w

Classification report for RandomForest pipeline:
                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      5047
               request       0.83      0.49      0.61      1115
                 offer       0.00      0.00      0.00        25
           aid_related       0.77      0.68      0.72      2740
          medical_help       0.62      0.06      0.11       516
      medical_products       0.94      0.10      0.18       312
     search_and_rescue       0.68      0.07      0.12       196
              security       0.00      0.00      0.00       126
              military       0.74      0.06      0.12       217
                 water       0.93      0.35      0.51       409
                  food       0.85      0.55      0.67       722
               shelter       0.81      0.37      0.50       578
              clothing       0.93      0.16      0.27        90
                 money       1.00      0.02      0.04 

c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _w

Classification report for AdaBoost pipeline:
                        precision    recall  f1-score   support

               related       0.81      0.97      0.88      5047
               request       0.72      0.50      0.59      1115
                 offer       0.00      0.00      0.00        25
           aid_related       0.77      0.58      0.66      2740
          medical_help       0.61      0.24      0.35       516
      medical_products       0.63      0.34      0.44       312
     search_and_rescue       0.67      0.22      0.33       196
              security       0.35      0.06      0.10       126
              military       0.57      0.32      0.41       217
                 water       0.76      0.64      0.69       409
                  food       0.82      0.66      0.73       722
               shelter       0.77      0.53      0.63       578
              clothing       0.72      0.49      0.58        90
                 money       0.53      0.34      0.41     

c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavio

In [14]:
#xgboost has the greatest f1 score so run grid search on this again to see what works best 
# number of folds reduced for computation time 
cv_verb = GridSearchCV(pipeline_verb_xgb, parameters, verbose=2, n_jobs=1,scoring='f1',cv = 2)
cv_verb.fit(X_train, y_train)

Fitting 2 folds for each of 15 candidates, totalling 30 fits


c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:523: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true

[CV] END classifier__estimator__learning_rate=0.01, classifier__estimator__n_estimators=50; total time= 1.8min


c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_scorer.py", line 376, in 

[CV] END classifier__estimator__learning_rate=0.01, classifier__estimator__n_estimators=50; total time= 3.7min


...

c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_scorer.py", line 376, in 

[CV] END classifier__estimator__learning_rate=0.2, classifier__estimator__n_estimators=1000; total time= 7.8min


c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_scorer.py", line 376, in 

[CV] END classifier__estimator__learning_rate=0.2, classifier__estimator__n_estimators=1000; total time= 7.9min


GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('count_vectorizer',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x000002606DD4F060>)),
                                                                                        ('tfidf_transformer',
                                                                                         TfidfTransformer())])),
                                                                       ('starting_verb',
                                                                        StartingVerbExtractor())])),
                                       ('classifier',
                                        MultiOutputClassifier(estimator=XGBClass...
                                                                                      max_depth=None,
                                                                                      max_leaves=None,
                                                                                      min_child_weight=None,
                                                                                      missing=nan,
                                                                                      monotone_constraints=None,
                                                                                      multi_strategy=None,
                                                                                      n_estimators=None,
                                                                                      n_jobs=None,
                                                                                      num_parallel_tree=None,
                                                                                      random_state=None, ...)))]),
             n_jobs=1,
             param_grid={'classifier__estimator__learning_rate': [0.01, 0.1,
                                                                  0.2],
                         'classifier__estimator__n_estimators': [50, 100, 200,
                                                                 500, 1000]},
             scoring='f1', verbose=2)

In [15]:
y_prediction_cv_verb= cv_verb.predict(X_test)
print(classification_report(y_test.values, y_prediction_cv_verb, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.77      1.00      0.87      5047
               request       0.00      0.00      0.00      1115
                 offer       0.00      0.00      0.00        25
           aid_related       0.82      0.32      0.46      2740
          medical_help       0.00      0.00      0.00       516
      medical_products       0.00      0.00      0.00       312
     search_and_rescue       0.00      0.00      0.00       196
              security       0.00      0.00      0.00       126
              military       0.00      0.00      0.00       217
                 water       0.00      0.00      0.00       409
                  food       0.00      0.00      0.00       722
               shelter       0.00      0.00      0.00       578
              clothing       0.00      0.00      0.00        90
                 money       0.00      0.00      0.00       158
        missing_people       0.00      

c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### 9. Export your model as a pickle file

In [17]:
import pickle

# Assuming pipeline_verb_xgb is your XGBoost pipeline
pipeline_verb_xgb.fit(X_train, y_train)
y_pred = pipeline_verb_xgb.predict(X_test)
print("Classification report for XGB pipeline:")
print(classification_report(y_test.values, y_pred, target_names=y.columns.values))

# Save the XGBoost pipeline to a file
with open('XGB_pipeline.pkl', 'wb') as file:
    pickle.dump(pipeline_verb_xgb, file)
print("XGB pipeline saved as XGB_pipeline.pkl")


c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:523: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Classification report for XGB pipeline:
                        precision    recall  f1-score   support

               related       0.84      0.94      0.89      5047
               request       0.78      0.55      0.65      1115
                 offer       0.00      0.00      0.00        25
           aid_related       0.78      0.65      0.71      2740
          medical_help       0.60      0.28      0.38       516
      medical_products       0.69      0.35      0.47       312
     search_and_rescue       0.73      0.19      0.31       196
              security       0.33      0.01      0.02       126
              military       0.60      0.33      0.43       217
                 water       0.77      0.67      0.71       409
                  food       0.81      0.73      0.77       722
               shelter       0.75      0.60      0.67       578
              clothing       0.75      0.52      0.61        90
                 money       0.61      0.31      0.41       158

c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\benedict.grant\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _w

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.